In [1]:
import numpy as np
import json
import os
import re
import collections
from sklearn.datasets.base import load_files
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import codecs
import pickle
from gensim.models.word2vec import LineSentence
from gensim.models.fasttext import FastText
import sys
from __future__ import division
from collections import OrderedDict
import pandas as pd
from sklearn.datasets import load_files
from collections import OrderedDict

In [2]:
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise.model_selection import KFold
from surprise import accuracy

In [3]:
def normalize_name(name):
    name = name.lower()
    name = re.sub(r"[.,\/#!$%\^\*;:{}=\_`~()@]", ' ', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name

In [4]:
path = '/home/irlab/Documents/recsys/mpd/data'
filenames = os.listdir(path)
mapping = {}
playlist_prob = OrderedDict()

In [5]:
#Train FastText Model for finding similarity
def train_playlist_embedding(filenames):    
    data = LineSentence('/home/irlab/files/data.txt')
    model = FastText(size=100)
    print('Building Vocab....')
    model.build_vocab(data)
    print('Training....')
    model.train(data, total_examples=model.corpus_count, epochs=model.iter)
    print(model)
    return model

In [6]:
#print(train_data)
model = train_playlist_embedding(filenames)

Building Vocab....
Training....


/home/irlab/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


FastText(vocab=9132, size=100, alpha=0.025)


In [7]:
#Find Similar Playlists with given name
def find_similar(playlist_name,model = model):
    names = []
    similar_playlists = model.wv.similar_by_word(playlist_name,20)
    similar_playlists = dict(similar_playlists)
    sim_playlist_name = [key for (key,value) in sorted(similar_playlists.items(),key=lambda x:x[1],reverse=True)]
    return sim_playlist_name

In [8]:
#Load mapping of playlist to filename
def load_filemapping():
    fp = open('/home/irlab/files/mappings/track_filename.json')
    mpd_mapping = json.load(fp)
    return mpd_mapping

In [9]:
mpd_mapping = load_filemapping()

In [10]:
#Load album, artist, track mappings
#Change the directory as per your system
fp = open('/home/irlab/files/stats/album_stats.json')
album_map = json.load(fp)
album_stat = sum(album_map.values())

fp = open('/home/irlab/files/stats/artist_stats.json')
artist_map = json.load(fp)
artist_stat = sum(artist_map.values())

fp = open('/home/irlab/files/stats/tracks_stats.json')
track_map = json.load(fp)
track_stat = sum(track_map.values())

print(album_stat,artist_stat,track_stat)

(66346428, 66346428, 66346428)


In [11]:
#Generate the ratings for all the tracks
def create_weighted_playlists(playlist_name,playlist_prob):
    try:
        similar_playlists = find_similar(playlist_name)
    except KeyError:
        temp = playlist_name.split()
        playlist_name = ''.join(temp)
        similar_playlists = find_similar(playlist_name)
        
    #print(similar_playlists)
    for similar_playlist in similar_playlists:
        if similar_playlist in mpd_mapping.keys():    
            #print(similar_playlist)
            name = mpd_mapping[similar_playlist]
            filename = os.sep.join((path,name))
            fp = open(filename,'r')
            playlist_file = json.load(fp)
            mpd_data = playlist_file['playlists']
            playlist_prob[similar_playlist] = {}
            for playlist in mpd_data:
                if normalize_name(playlist['name']) in similar_playlist:
                    for track in playlist['tracks']:
                        p1 = artist_map[track['artist_uri']]/artist_stat
                        p2 = album_map[track['album_uri']]/album_stat
                        p3 = track_map[track['track_uri']]/track_stat
                        playlist_prob[similar_playlist][track['track_uri']] = (p1+p2+p3)/(1+track['pos'])
                        #print(playlist_prob[similar_playlist])
    return playlist_prob

In [12]:
#Create Rating matrix
def create_matrix(playlist_prob):
    pd.DataFrame(data=playlist_prob,dtype=np.int64)
    df = pd.DataFrame(data=playlist_prob)
    df = df.fillna(0)
    df = df.T
    return df

In [13]:
#Load data in surprise api format
def load_data(tracks,playlists,ratings):
    ratings_dict = {'itemID': tracks,
                    'userID': playlists,
                    'rating': ratings}
    df1 = pd.DataFrame(ratings_dict)
    reader = Reader(rating_scale=(0,1))
    data = Dataset.load_from_df(df1[['userID','itemID','rating']],reader)
    #print(df1)
    return data

In [14]:
#Train data using SVD and gradient descent
def train(data):
    kf = KFold(n_splits=12)
    algo = SVD(n_epochs=55,lr_all=0.006,reg_all=0.075)
    for trainset,testset in kf.split(data):
        algo.fit(trainset)
        prediction = algo.test(testset)
        print(accuracy.rmse(prediction,verbose=True))
    return algo

In [15]:
#Predict ratings for tracks
def test(playlist_name,tracks,algo,seed_tracks):
    tracks_pred = [n for n in tracks if n not in seed_tracks]
    Recommend = {}
    for track in tracks_pred:
        predict = algo.predict(playlist_name,track)
        Recommend[track] = predict[3]
    recommend_list = [key for (key,value) in sorted(Recommend.items(),key=lambda x:x[1],reverse=True)]
    return recommend_list

In [16]:
#Get tracks if recommendations less than 500
def random_tracks(recommend_list,seed_tracks):
    tracks = [key for (key,value) in sorted(track_map.items(),key=lambda x:x[1],reverse=True)]
    extend_tracks = [track for track in tracks if track not in recommend_list]
    extend_tracks = [track for track in extend_tracks if track not in seed_tracks]
    diff = 500 - len(recommend_list)
    print(len(extend_tracks[0:diff]))
    for name in extend_tracks[0:diff]:
        print(name)
        recommend_list.append(name)
    return recommend_list

In [17]:
#Change this as per your directory
path1 = '/home/irlab/Documents/recsys/challenge'
challenge_file = 'challenge_set.json'
filename = os.sep.join((path1,challenge_file))
fp = open(filename)
challenge_set = json.load(fp)
playlists = challenge_set['playlists']
dict_playlist = {}

In [18]:
import csv

In [21]:
#string = ['team_info','main','Dark_Horse_1','201711040@daiict.ac.in']

#with open('/home/irlab/Documents/recsys/mpd/codes/output/output2.csv','w') as csvfile:
#        filewriter = csv.writer(csvfile,delimiter = ',')
#        filewriter.writerow(string)

count=0
for playlist in playlists:
    list_temp = []
    seed_tracks = []
    recommend_list = []
    if 'name' in playlist.keys() and playlist['num_samples']==25:
        pid = playlist['pid']
        list_temp.insert(0,pid)
        count+=1
        print(count,playlist['name'])
        if count >= 917:
            tracks = playlist['tracks']
            playlist_prob.clear()
            playlist_prob[normalize_name(playlist['name'])] = {}
            i=0
            for track in tracks:
                if track['artist_uri'] not in artist_map.keys():
                    p1 = 1
                else:    
                    p1 = artist_map[track['artist_uri']]/artist_stat
                if track['album_uri'] not in artist_map.keys():
                    p2 = 1
                else:
                    p2 = album_map[track['album_uri']]/album_stat
                if track['track_uri'] not in artist_map.keys():
                    p3 = 1
                else:
                    p3 = track_map[track['track_uri']]/track_stat
                seed_tracks.insert(i,track['track_uri'])
                playlist_prob[normalize_name(playlist['name'])][track['track_uri']] = (p1+p2+p3)/(1+track['pos']) 
                i+=1
            #print(seed_tracks)
            print('Finding Similar Playlists...')
            playlist_prob = create_weighted_playlists(normalize_name(playlist['name']),playlist_prob)
            print('Creating Rating matrix...')
            df = create_matrix(playlist_prob)
            column_names = list(df.columns)
            row_names = list(df.index)
            matrix = df.values
            row,col = np.nonzero(matrix)
            row = row.tolist()
            col = col.tolist()
            n_tracks = [column_names[i] for i in col]
            playlists = [row_names[i] for i in row]
            ratings = [matrix[r][c] for (r,c) in zip(row,col)]
            print('Loading data for training')
            data = load_data(n_tracks,playlists,ratings)
            print('Training....')
            algo = train(data)
            print('Predicting....')
            recommend_list = test(normalize_name(playlist['name']),n_tracks,algo,seed_tracks)
            print('Playlist Size:',len(recommend_list))
            if len(recommend_list[0:500])<500:
                print('Picking popular tracks...')
                recommend_list = random_tracks(recommend_list,seed_tracks)
                print('New Playlist Size:',len(recommend_list))
            list_temp.extend(recommend_list[0:500])
            print('Writing to file.....')
            #Change this as per your directory
            with open('/home/irlab/Documents/recsys/mpd/codes/output/final_submission.csv','a') as csvfile1:
                filewriter = csv.writer(csvfile1,delimiter = ',')
                filewriter.writerow(list_temp)

(1, u'Oldies')
(2, u'Study Music')
(3, u'Music i like.')
(4, u'Disney')
(5, u'Rock')
(6, u'EVERYTHING')
(7, u'new new')
(8, u'ghetto')
(9, u'game day')
(10, u'new country')
(11, u'Indie')
(12, u'Film Scores')
(13, u'Special')
(14, u'peach')
(15, u'Prom 2k17')
(16, u'workout')
(17, u'Main')
(18, u'mine')
(19, u'get low')
(20, u'pop')
(21, u'Classic Jams')
(22, u'En Espa\xf1ol')
(23, u'Chillwave')
(24, u'Throwback Jamz')
(25, u'Rock')
(26, u'trending ')
(27, u'Hip Hop')
(28, u'Upbeat')
(29, u'Chillin')
(30, u'White Girl Music')
(31, u'You know')
(32, u'Classic Rock')
(33, u'Classic Rock')
(34, u'everyday jams')
(35, u'Albums')
(36, u'good stuff')
(37, u'love it')
(38, u'Cloud 9')
(39, u'work')
(40, u'Spoopy')
(41, u'Firefly')
(42, u'Camp')
(43, u'My favorites')
(44, u'jam (:')
(45, u'Da best')
(46, u'Party')
(47, u'Alternative/Pop')
(48, u'Classics')
(49, u'Roadtrip')
(50, u'Oldies')
(51, u'2013/2014')
(52, u'throwback')
(53, u'yes')
(54, u'New')
(55, u'trap')
(56, u'Blast from the Past'

Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0578
0.05779953913867815
RMSE: 0.0144
0.014423342531588727
RMSE: 0.0180
0.018042216580634107
RMSE: 0.0085
0.008519215602807497
RMSE: 0.0108
0.010797844984279055
RMSE: 0.0224
0.022350361587757286
RMSE: 0.0313
0.03134185685222544
RMSE: 0.0132
0.013191585816732916
RMSE: 0.1375
0.13747932161348497
RMSE: 0.0126
0.012570783252896353
RMSE: 0.0069
0.006921834117825581
RMSE: 0.0134
0.013413733878454981
Predicting....
('Playlist Size:', 1937)
Writing to file.....
(918, u'Dirt')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0164
0.016418927143120002
RMSE: 0.1561
0.15606469646267443
RMSE: 0.0028
0.002789680453404651
RMSE: 0.0107
0.010711973989413672
RMSE: 0.0660
0.06600412219020549
RMSE: 0.0298
0.02983399646968588
RMSE: 0.0146
0.014583466959095327
RMSE: 0.0194
0.01936148601213654
RMSE: 0.0177
0.01773050411757742
RMSE: 0.0065
0.006531970515416

RMSE: 0.1497
0.14965551024131893
RMSE: 0.0066
0.0065551875837981674
RMSE: 0.0098
0.009783862077213
RMSE: 0.0634
0.06343955887691007
RMSE: 0.0131
0.013099878724120734
RMSE: 0.0177
0.017744270157157307
RMSE: 0.0145
0.014536181399899784
RMSE: 0.0325
0.03251152896415423
RMSE: 0.0280
0.02801806094033202
RMSE: 0.0064
0.006404719547153836
RMSE: 0.0465
0.04651366805243063
RMSE: 0.0406
0.04055406324327313
Predicting....
('Playlist Size:', 1600)
Writing to file.....
(932, u'nasheed')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0400
0.03995294382540251
RMSE: 0.0029
0.002898497562687989
RMSE: 0.0781
0.07810362900190354
RMSE: 0.0164
0.01635953622542302
RMSE: 0.0166
0.016570367699569415
RMSE: 0.0171
0.017106255623021795
RMSE: 0.0524
0.05238632220869584
RMSE: 0.0140
0.013955266257541937
RMSE: 0.0036
0.0036172420691795515
RMSE: 0.0134
0.013359085814097133
RMSE: 0.1821
0.18212978120674245
RMSE: 0.0152
0.015246335241277047
Predicting....
('Playli

RMSE: 0.0199
0.01993835049854714
RMSE: 0.0129
0.012866251144683906
RMSE: 0.0220
0.021999189068895197
RMSE: 0.0746
0.07457635842096264
RMSE: 0.0181
0.01810764895565233
RMSE: 0.0293
0.029252664854946615
RMSE: 0.1832
0.1831898951184568
RMSE: 0.0347
0.03466758425181406
RMSE: 0.0146
0.01463842045563651
Predicting....
('Playlist Size:', 1190)
Writing to file.....
(946, u'Cha Cha')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0960
0.0960287626230832
RMSE: 0.0309
0.03086706533072028
RMSE: 0.0567
0.05666120463846867
RMSE: 0.0265
0.02654113511957481
RMSE: 0.0180
0.0180058073954556
RMSE: 0.0317
0.03172561810497863
RMSE: 0.0056
0.005551005911567554
RMSE: 0.0216
0.021623924018512958
RMSE: 0.0382
0.038150168022556996
RMSE: 0.0350
0.035008000204168936
RMSE: 0.2196
0.21958606365130584
RMSE: 0.0041
0.0041353453699979476
Predicting....
('Playlist Size:', 753)
Writing to file.....
(947, u'Lift')
Finding Similar Playlists...
Creating Rating matrix..

RMSE: 0.0507
0.05072615470526084
RMSE: 0.0207
0.020697885781665027
RMSE: 0.2122
0.21224486170585585
RMSE: 0.0234
0.023397339605897134
Predicting....
('Playlist Size:', 823)
Writing to file.....
(960, u'good')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0019
0.00190666348306745
RMSE: 0.0241
0.024082191319130808
RMSE: 0.0183
0.01828490727414761
RMSE: 0.0192
0.019182920625872523
RMSE: 0.0111
0.011051422401995992
RMSE: 0.1189
0.1189421835707227
RMSE: 0.0583
0.05830058557431889
RMSE: 0.0040
0.0040176431309157734
RMSE: 0.0091
0.009117858997488616
RMSE: 0.0115
0.01152920379820818
RMSE: 0.0178
0.017835956109748828
RMSE: 0.0100
0.010010388637635858
Predicting....
('Playlist Size:', 2537)
Writing to file.....
(961, u'oldies')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0095
0.009451415364589115
RMSE: 0.0076
0.007563831493479395
RMSE: 0.0125
0.012468477038438053
RMSE: 0.0494
0.04935

...

RMSE: 0.0044
0.004442477594407078
RMSE: 0.0011
0.0010952597153809545
RMSE: 0.0053
0.005344497946225559
RMSE: 0.0029
0.0029317083103365454
Predicting....
('Playlist Size:', 1572)
Writing to file.....
(1933, u'partyyyy')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0032
0.003165863092037118
RMSE: 0.0044
0.004355991504206949
RMSE: 0.0016
0.0016118844106645543
RMSE: 0.0035
0.0034834731509219593
RMSE: 0.0064
0.00641418441816448
RMSE: 0.0017
0.001654413372578312
RMSE: 0.0043
0.0042508447069370585
RMSE: 0.0021
0.002118711391635363
RMSE: 0.0016
0.0015708486955166095
RMSE: 0.0063
0.006275955731671632
RMSE: 0.0071
0.007121464374540466
RMSE: 0.0717
0.07174017982820832
Predicting....
('Playlist Size:', 4807)
Writing to file.....
(1934, u'ARUBA')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0257
0.025708138061730827
RMSE: 0.0069
0.006930220497228365
RMSE: 0.0200
0.019993769771121994
RMS

RMSE: 0.0058
0.00579683058944336
RMSE: 0.0102
0.010249977730651564
RMSE: 0.0046
0.004579881198111976
RMSE: 0.0035
0.0034549613911471918
Predicting....
('Playlist Size:', 2094)
Writing to file.....
(1947, u'FUNK')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0125
0.012499176815094602
RMSE: 0.0047
0.0047339890503274975
RMSE: 0.0135
0.013496593330102972
RMSE: 0.0224
0.022442865427355255
RMSE: 0.0099
0.009933049603741568
RMSE: 0.0188
0.01876368246945967
RMSE: 0.1598
0.15977927157220978
RMSE: 0.0204
0.02044243708336195
RMSE: 0.0111
0.011141787338422526
RMSE: 0.0101
0.010110666037774437
RMSE: 0.0753
0.07525047934112158
RMSE: 0.0059
0.005893013909723383
Predicting....
('Playlist Size:', 1570)
Writing to file.....
(1948, u'beatz ')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0077
0.007749377086734958
RMSE: 0.0037
0.0037158461043356595
RMSE: 0.0037
0.003689007738242313
RMSE: 0.0099

RMSE: 0.0052
0.005221731466318741
RMSE: 0.0145
0.014453603192656313
Predicting....
('Playlist Size:', 1342)
Writing to file.....
(1961, u'OSL')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0089
0.008919793287154448
RMSE: 0.0084
0.008424154974671066
RMSE: 0.0098
0.009787593148761715
RMSE: 0.0035
0.0035030376058916906
RMSE: 0.0071
0.007085493289223467
RMSE: 0.0063
0.006254979373608485
RMSE: 0.0032
0.003156861723066281
RMSE: 0.0038
0.0038252903049778115
RMSE: 0.0027
0.0027037068956854136
RMSE: 0.0103
0.010349522513375564
RMSE: 0.0573
0.05731399408227879
RMSE: 0.0084
0.008393654102698817
Predicting....
('Playlist Size:', 1615)
Writing to file.....
(1962, u'playlist ')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0063
0.006322426596178226
RMSE: 0.0108
0.010827508021377101
RMSE: 0.0119
0.011883150805403293
RMSE: 0.0073
0.007317078407717516
RMSE: 0.0141
0.01413294962645594
RMSE: 0

RMSE: 0.0041
0.004129729119924317
Predicting....
('Playlist Size:', 3659)
Writing to file.....
(1975, u'march madness')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0111
0.01110018884705278
RMSE: 0.0072
0.007185462877176897
RMSE: 0.0122
0.012168450191529756
RMSE: 0.1552
0.15520775584670057
RMSE: 0.0096
0.009550728526625614
RMSE: 0.0105
0.010543973769356679
RMSE: 0.0076
0.007591274592827882
RMSE: 0.0028
0.002799686285579407
RMSE: 0.0069
0.006881822878390733
RMSE: 0.0090
0.009018349575288494
RMSE: 0.0067
0.006660656396361894
RMSE: 0.0097
0.009735232318278901
Predicting....
('Playlist Size:', 1829)
Writing to file.....
(1976, u'Country')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0126
0.012585198135271624
RMSE: 0.0145
0.014479494003658487
RMSE: 0.0067
0.006748485147214494
RMSE: 0.0121
0.012076616163973188
RMSE: 0.0067
0.006719415444418895
RMSE: 0.0065
0.006542906677047567
RM

Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0130
0.01299779430072756
RMSE: 0.0144
0.014351981172056532
RMSE: 0.0088
0.008784284252296574
RMSE: 0.0122
0.012153039598893125
RMSE: 0.0105
0.010494982612612625
RMSE: 0.0061
0.00608068050306864
RMSE: 0.0083
0.008303330417894427
RMSE: 0.0129
0.012938272007299566
RMSE: 0.0153
0.015349148837729133
RMSE: 0.1716
0.17157028080109238
RMSE: 0.0160
0.01595013152556994
RMSE: 0.0110
0.010999461547513977
Predicting....
('Playlist Size:', 1493)
Writing to file.....
(1990, u'Rock')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0016
0.0015531938121983
RMSE: 0.0012
0.0012415776429112318
RMSE: 0.0054
0.005404128357527677
RMSE: 0.0016
0.001620979289565436
RMSE: 0.0011
0.001094819175388956
RMSE: 0.0030
0.00304517582337894
RMSE: 0.0021
0.0020598009435422314
RMSE: 0.0018
0.0017817928377068123
RMSE: 0.0011
0.001075661762978737
RMSE: 0.0015
0.0014635875

1006485
